In [ ]:
# imports
# suppress holoviews warning. Using warnings module didn't work
import logging
# logging.getLogger("param.Dimension").setLevel(logging.CRITICAL)
# logging.getLogger("param.ParameterizedMetaclass").setLevel(logging.CRITICAL)
# logging.getLogger("param.HistogramPlot").setLevel(logging.CRITICAL)
# logging.getLogger("param.AdjointLayout").setLevel(logging.CRITICAL)
# logging.getLogger("param.OverlayPlot").setLevel(logging.CRITICAL)
# logging.getLogger("param.HoloMap").setLevel(logging.CRITICAL)
# logging.getLogger("param.CurvePlot").setLevel(logging.CRITICAL)
# logging.getLogger("param.Layout").setLevel(logging.CRITICAL)
# logging.getLogger("param.LayerPlot").setLevel(logging.CRITICAL)
# logging.getLogger("param.RasterPlot").setLevel(logging.CRITICAL)
# logging.getLogger("param.Scatter3DPlot").setLevel(logging.CRITICAL)
import os
import sys
sys.path.insert(0, os.path.abspath(r'D:\Code Repos\prey_capture'))


import panel as pn
import holoviews as hv
from holoviews import opts, dim
hv.extension('bokeh')
from bokeh.resources import INLINE

import paths
import functions_bondjango as bd
import functions_misc as fm
import functions_plotting as fp
import pandas as pd
import numpy as np
import sklearn.mixture as mix
import sklearn.decomposition as decomp
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn import svm, datasets
from sklearn import preprocessing
import sklearn.linear_model as lin
import sklearn.metrics as smet
import scipy.signal as ss
import scipy.stats as stat
import scipy.spatial.distance as space

import random
import functions_data_handling as fd
import functions_vame as fv
import importlib
import processing_parameters
import PSID
from PSID.evaluation import evalPrediction
import sklearn.cross_decomposition as cros
import umap


In [ ]:
# Load the desired files
importlib.reload(processing_parameters)

# define the threshold for matched cells
match_threshold = 10
# get the data paths
try: 
    data_path = snakemake.input[0]
except NameError:
    # get the search list
    search_list = processing_parameters.search_list
    # allocate memory for the data
    pre_normal_data = []
    pre_mod_data = []
    
    # allocate a list for all paths (need to preload to get the dates)
    all_paths = []
    # for all the search strings
    for search_string in search_list:

        # query the database for data to plot
        data_all = bd.query_database('analyzed_data', search_string)
        data_all = [el for el in data_all if 'preproc' in el['slug']]
        data_path = [el['analysis_path'] for el in data_all if '_preproc' in el['slug']]
        all_paths.append(data_path)
    print(all_paths[0])
    # get the dates present
    data_dates = np.unique([os.path.basename(el)[:10] for el in np.concatenate(all_paths)])
    # now load the files
    for data_path in all_paths:
        # load the calcium data
        beh_data = []
        # for all the files
        for files in data_path:
            # load the data
            with pd.HDFStore(files) as h:
                beh_data.append(h['full_traces'])
                if '/matched_calcium' in h.keys():
                    # get the cell matches
                    cell_matches = h['cell_matches']
                    
                    # perform only if there are more files
                    if len(data_dates) > 1:
                        match_dates = [el for el in data_dates if el in cell_matches.columns]
                        # get only the days present in the search
                        cell_matches = cell_matches[match_dates]
                        # generate a list with the number of days and the number of cells kept

                        # get the unique cell combinations
                        # unique contains the unique patterns followed by cells across days
                        # inverse indicates which pattern is followed by each cell
                        # count contains the number of times each pattern is found
                        unique, inverse, counts = np.unique(~np.isnan(cell_matches.to_numpy()), axis=0, 
                                        return_counts=True, return_inverse=True)
                        
                        # remove the single day and no day cases
                        counts[np.sum(unique, axis=1)==0] = 0
                        counts[np.sum(unique, axis=1)==1] = 0

                        # get an index vector with only the most popular pattern
                        # (regardless of how many cells share it)
                        cell_idx = np.array(inverse==np.argmax(counts))
                        # leave only those cells in the matches
                        cell_matches = cell_matches.iloc[cell_idx, :]
                    else:
                        counts = 1
                        cell_idx = cell_matches[data_dates].to_numpy()
                        cell_idx = ~np.isnan(cell_idx)
                        cell_matches = cell_matches.iloc[cell_idx, :]
                    # separate based on normal vs mod
                    if 'dark' in files:
                        pre_mod_data.append((files, h['matched_calcium'], cell_matches))
                    else:
                        pre_normal_data.append((files, h['matched_calcium'], cell_matches))
                    
print(f'Number of matched cells: {np.sum(cell_idx)}')
print(f'Number of matched days: {unique[np.argmax(counts)].sum()}')

# hv.Image(cell_matches.to_numpy()).opts(width=800, tools=['hover'])

In [ ]:
# Leave only common cells across all datasets

# allocate memory for the cleaned up data
normal_data = []
mod_data = []
plot_data = []

# print(pre_normal_data[0][2])
# for all the normal trials
for idx, el in enumerate(pre_normal_data):
    # get the date
    current_date = os.path.basename(el[0])[:10]
    # get the corresponding indexes
    current_idx = el[2][current_date].to_numpy()
    # if they're all nans, skip the day
    if np.isnan(np.sum(current_idx)):
        continue
    # get the current df
    current_df = el[1]
    labels = list(current_df.columns)
    cells = [el for el in labels if 'cell' in el]
    not_cells = [el for el in labels if 'cell' not in el]
    # get the non-cell data
    non_cell_data = current_df[not_cells]
    # get the current calcium data
    cell_data = current_df[cells]
    # remove the non matched cells
#     cell_data = cell_data.iloc[:, current_idx]
    # rename the cell fields
    cell_names = ['cell_' + str(el) for el in np.arange(cell_data.shape[1])]
    cell_data.columns = cell_names
    # normalize the single trial activity
#     cell_data = (cell_data-cell_data.mean())/cell_data.std()
    # get rid of the nan values created by silent neurons in a given trial
    cell_data[np.isnan(cell_data)] = 0
    # assemble a new data frame with only the matched cells and the rest of the data
    normal_data.append(pd.concat((non_cell_data, cell_data), axis=1))
    
print(normal_data[0].shape)
print(normal_data[0].columns)

In [ ]:
# align coordinates egocentrically
importlib.reload(fv)
# define the relevant variables
crop_size = (200, 200)

# allocate memory for the new coordinates
new_coordinates = []
new_columns = []
# for all trials
for trial in normal_data:
    # run the alignment
    coordinates, _, column_list = fv.align_demo(trial, [], [], [],
                   crop_size, use_video=False, check_video=False, vid_path=None)
    # save
    new_coordinates.append(coordinates)
    new_columns.append(column_list)

In [ ]:
# Replace the raw coordinates by the egocentric ones
print(new_columns[0])

# for all the trials
for idx, trial in enumerate(normal_data):
    # replace the columns
    for idx2, column in enumerate(new_columns[idx]):
        trial[column] = new_coordinates[idx][idx2, :]

In [ ]:
# Multivariate regression


# define the target variable
# target_behavior = ['cricket_0_x', 'cricket_0_y', 'cricket_0_mouse_distance']
# target_behavior = ['cricket_0_x']
# target_behavior = ['mouse_head_x', 'mouse_head_y', 'mouse_speed']
# target_behavior = ['mouse_x']
# target_behavior = ['cricket_0_visual_angle']
# target_behavior = ['cricket_0_mouse_distance', 'cricket_0_delta_heading', 'cricket_0_visual_angle']
# target_behavior = ['mouse_speed']
# target_behavior = ['cricket_0_delta_heading']
target_behavior = ['cricket_0_mouse_distance']

# define the downsampling factor
downsamp = 1
# get the number of points for each trial
trial_vector = np.vstack([np.ones((el.shape[0], 1))*idx for idx, el in enumerate(normal_data)]).flatten()

# get the table
sub_data = pd.concat(normal_data)
print(f'Shape of the input data (frames*columns): {sub_data.shape}')

# get the available columns
labels = list(sub_data.columns)
cells = [el for el in labels if 'cell' in el]
not_cells = [el for el in labels if 'cell' not in el]
# get the cell data
calcium_data = np.array(sub_data[cells].copy())

# # use the PSID calculated latent
# calcium_data = np.concatenate(final_latent, axis=0)

# generate design matrix with past time points too
# define the number of time points to accumulate
time_points = 1

if time_points > 1:
    # allocate memory for the matrix
    design_matrix = np.zeros((calcium_data.shape[0], calcium_data.shape[1]*time_points))
    # create the source matrix
#     source_matrix = np.vstack((np.zeros((time_points, calcium_data.shape[1])), calcium_data))
    source_matrix = np.vstack((np.zeros((time_points-1, calcium_data.shape[1])), calcium_data))

    # for all the timepoints
    for times in np.arange(calcium_data.shape[0]):
        design_matrix[times, :] = source_matrix[times:times+time_points, :].flatten()

    # replace the calcium data
    calcium_data = design_matrix

# subselect cells based on responsiveness
# define the fraction of cells from the top responsive to take
cell_fraction = 1
# if not all cells are required
if cell_fraction < 1:
    # get the responsiveness (as average zscored activity)
    responsiveness = np.mean(stat.zscore(calcium_data, axis=0), axis=0)
    # get the desired percentile
    target_percentile = np.percentile(responsiveness, (1-cell_fraction)*100)
    # take only the selected cells
    calcium_data = calcium_data[:, responsiveness>target_percentile]

# get the parameter
parameter = sub_data.loc[:, target_behavior].to_numpy()
# smooth the parameter
parameter[:, 0] = ss.medfilt(parameter[:, 0], 21)

# define the time shift (in frames, positive is to the future)
time_shift = 0
# trim the parameter and calcium traces accordingly
if time_shift > 0:
    parameter = parameter[time_shift:]
    calcium_data = calcium_data[:-time_shift, :]
    trial_vector = trial_vector[:-time_shift]
elif time_shift < 0:
    parameter = parameter[:time_shift]
    calcium_data = calcium_data[-time_shift:, :]
    trial_vector = trial_vector[-time_shift:]


# exclude points without the parameter
nan_vector = np.sum(np.isnan(parameter), axis=1) == 0
parameter = parameter[nan_vector, :]
calcium_data = calcium_data[nan_vector, :]
trial_vector = trial_vector[nan_vector]

# bin the data
if downsamp > 1:
    parameter = ss.decimate(parameter, downsamp, axis=0)
    calcium_data = ss.decimate(calcium_data, downsamp, axis=0)
    # warning, ghetto implementation
    trial_vector = trial_vector[::downsamp]

# # turn the parameter into labels
# parameter = np.abs(sub_data.loc[:, 'cricket_0_mouse_distance'].to_numpy()) > 25
# parameter = np.expand_dims(parameter[nan_vector], axis=1)
# print(np.unique(parameter, return_counts=True))
# print('Warning, using delta heading as classification labels')
    
# # remove the repeated values in a row (from tracking)
# keep_vector = np.pad(np.diff(parameter[:, 0])!=0, (1, 0), mode='constant', constant_values=0)

# # limit the data to a field of view
# sec_parameter = sub_data.loc[:, 'cricket_0_delta_heading'].to_numpy()
# sec_parameter = sec_parameter[nan_vector]
# parameter_range = (-100 < sec_parameter) & (sec_parameter < 100)
# keep_vector = parameter_range
# keep_vector = (keep_vector) & (parameter_range)

# # limit the data to a given distance to prey
# distance_to_prey = sub_data.loc[:, 'cricket_0_mouse_distance'].to_numpy()
# distance_to_prey = distance_to_prey[nan_vector]
# distance_vector = distance_to_prey > 3
# keep_vector = (keep_vector) & (distance_vector)

# # select the data based on the above criteria
# parameter = parameter[keep_vector, :]
# calcium_data = calcium_data[keep_vector, :]
# trial_vector = trial_vector[keep_vector]

# # bin the parameter vector
# parameter = np.digitize(parameter, np.linspace(np.min(parameter), np.max(parameter), 10)) - 1

# # shufle the label vector
# random.shuffle(parameter)

# split the data
calcium_train, calcium_test, parameter_train, parameter_test = \
    train_test_split(calcium_data, parameter, test_size=0.2, shuffle=False)

# scale the features
calcium_scaler = preprocessing.StandardScaler().fit(calcium_train)
calcium_train = calcium_scaler.transform(calcium_train)
calcium_test = calcium_scaler.transform(calcium_test)

# parameter_scaler = preprocessing.StandardScaler().fit(parameter_train)
# parameter_train = parameter_scaler.transform(parameter_train)
# parameter_test = parameter_scaler.transform(parameter_test)

# initialize the classifier
# linear = lin.MultiTaskElasticNetCV(max_iter=5000, l1_ratio=[.1, .5, .7, .9, .95, .99, 1], 
#                                    n_jobs=7, alphas=[.0001, .01, .1, 1, 10, 100], fit_intercept=True)
#                                    n_jobs=7, alphas=[.001])
linear = lin.TweedieRegressor(alpha=0.01, max_iter=5000, fit_intercept=False, power=0)
# linear = lin.HuberRegressor(alpha=.1, max_iter=5000, fit_intercept=True, epsilon=10)
# linear = svm.SVR(max_iter=10000, kernel='rbf', C=100)
# linear = svm.LinearSVR(max_iter=5000, C=100)
# linear = svm.LinearSVC(max_iter=100000, C=1000, class_weight='balanced')
# linear = svm.SVC(max_iter=10000, C=.1, class_weight='balanced', kernel='rbf')

# train the classifier
linear.fit(calcium_train, parameter_train)

# predict train and test
linear_pred = linear.predict(calcium_train)
linear_pred_last = linear.predict(calcium_test)

# # smooth the predictions
linear_pred = ss.medfilt(linear_pred.flatten(), 21)
linear_pred_last = ss.medfilt(linear_pred_last.flatten(), 21)

try:
#     print('Train Exp variance:'+str(smet.r2_score(parameter_train, linear_pred)))
#     print('Test Exp variance:'+str(smet.r2_score(parameter_test, linear_pred_last)))
    print('Train Correlation:'+str(stat.spearmanr(parameter_train.flatten(), linear_pred)[0]))
    print('Test Correlation:'+str(stat.spearmanr(parameter_test.flatten(), linear_pred_last)[0]))
#     print('Exp variance:'+str(smet.r2_score(parameter, linear_pred)))
#     print('Exp variance:'+str(smet.r2_score(parameter_last, linear_pred_last)))
except TypeError:
    print('Train Accuracy:'+str(smet.accuracy_score(parameter_train, linear_pred)))
    print('Test Accuracy:'+str(smet.accuracy_score(parameter_test, linear_pred_last)))
#     print('Accuracy:'+str(smet.accuracy_score(parameter, linear_pred)))
#     print('Accuracy:'+str(smet.accuracy_score(parameter_last, linear_pred_last)))

In [ ]:
hv.extension('bokeh')
# plot the weights
# weights = linear.coef_
# print(weights)
# freq, edges = np.histogram(weights)
# barplot = hv.Bars((edges, freq)).opts(width=800)

train_scatter = hv.Scatter((parameter_train[:, 0], linear_pred), 
                           kdims=['Data', 'Prediction']).opts(width=400, height=400)
test_scatter = hv.Scatter((parameter_test[:, 0], linear_pred_last),
                          kdims=['Data', 'Prediction']).opts(width=400, height=400)


# plot the predictions

# x_real = np.array(range(parameter.shape[0]))
# real = hv.Curve((x_real, parameter[:, 0])).opts(width=800)
x_real = np.array(range(parameter_train.shape[0]))
real = hv.Curve((x_real, parameter_train[:, 0])).opts(width=800)

# x_last = np.array(range(parameter_last.shape[0]))
# last = hv.Curve((x_last, parameter_last[:, 0])).opts(width=800)
x_last = np.array(range(parameter_test.shape[0]))
last = hv.Curve((x_last, parameter_test[:, 0])).opts(width=800)

try:
    pred_last = hv.Curve((x_last, linear_pred_last[:, 0]))
except IndexError:
    pred_last = hv.Curve((x_last, linear_pred_last))
try:
    pred_real = hv.Curve((x_real, linear_pred[:, 0]))
except IndexError:
    pred_real = hv.Curve((x_real, linear_pred))

# combo = (train_scatter+test_scatter)
# combo

(real*pred_real+last*pred_last+train_scatter+test_scatter).opts(shared_axes=False).cols(1)

In [ ]:
# custom aggregator for hextiles
def aggregator(data_in):
    if data_in.shape[0] < 3:
        return 0
    else:
        return np.nanmean(data_in)

In [ ]:
# calculate a rolling CC and compare to target variables

# define the window width (in frames)
window = 10
half_window = int(window/2)
# define the target variable
target_variable0 = 'cricket_0_delta_heading'
target_variable1 = 'cricket_0_mouse_distance'

# target_variable0 = 'mouse_x'
# target_variable1 = 'mouse_y'

# target_variable0 = 'cricket_0_x'
# target_variable1 = 'cricket_0_y'

# allocate memory for the rolling correlation
rolling_correlation = np.zeros_like(parameter)
# generate the prediction
prediction = ss.medfilt(linear.predict(calcium_scaler.transform(calcium_data)).flatten(), 21)
# for all the elements within the window
for idx, _ in enumerate(prediction[half_window:-half_window]):
    # get the correlation coefficient for the prediction window and store
    correlation = stat.spearmanr(parameter[idx-half_window:idx+half_window], prediction[idx-half_window:idx+half_window])[0]
    rolling_correlation[idx+half_window] = correlation

# produce the average calcium activity
average_calcium = np.nanmean(calcium_data, axis=1)

# define the plotting variables
x1 = sub_data.loc[:, target_variable0]
# y0 = rolling_correlation.flatten()
y1 = sub_data.loc[:, target_variable1]

# angle_vector = x1 < 0

# x1 = x1[angle_vector]
# y1 = y1[angle_vector]
# rolling_correlation = rolling_correlation[angle_vector]
# convert to polar coordinates
# x1_pol = y1*np.cos(np.deg2rad(x1))
# y1_pol = y1*np.sin(np.deg2rad(x1))
# y1 = x1_pol
# x1 = y1_pol
# hextile parameters
gridsize = 20
min_count = 3
# plot the comparison
plot0 = hv.HexTiles((x1, y1), kdims=[target_variable0, 'Correlation'])
plot0.opts(width=700, height=700, tools=['hover'], gridsize=gridsize, min_count=min_count)

plot1 = hv.HexTiles((x1, y1, rolling_correlation.flatten()), 
                   kdims=[target_variable0, target_variable1], vdims=['Correlation'])
plot1.opts(width=700, height=700, aggregator=aggregator, tools=['hover'], gridsize=gridsize, cmap='RdBu', clim=(-1, 1))

# plot1 = hv.Scatter((x1, y1, rolling_correlation.flatten()), 
#                    kdims=[target_variable0, target_variable1], vdims=['Correlation'])
# plot1.opts(width=700, height=700, tools=['hover'], cmap='RdBu')

plot2 = hv.HexTiles((x1, y1, average_calcium.flatten()), 
                   kdims=[target_variable0, target_variable1], vdims=['Activity'])
plot2.opts(width=700, height=700, aggregator=aggregator, tools=['hover'], gridsize=gridsize)


(plot0+plot1+plot2).opts(shared_axes=False).cols(2)

In [ ]:
# plot the data going into the regression
hv.extension('bokeh')

x_train = np.array(range(parameter_train.shape[0]))
x_test = np.array(range(parameter_test.shape[0]))

calcium_train_plot = hv.Image(calcium_train.T)
calcium_train_plot.opts(width=1200, tools=['hover'])
parameter_train_plot = hv.Curve((x_train, parameter_train[:, 0]))
parameter_train_plot.opts(width=1200, tools=['hover'])
calcium_test_plot = hv.Image(calcium_test.T)
calcium_test_plot.opts(width=1200, tools=['hover'])
parameter_test_plot = hv.Curve((x_test, parameter_test[:, 0]))
parameter_test_plot.opts(width=1200, tools=['hover'])



(calcium_train_plot+parameter_train_plot+calcium_test_plot+parameter_test_plot).opts(shared_axes=False).cols(1)


In [ ]:
hv.extension('bokeh')
# plot the PCA trajectories of the data
pca_model = decomp.PCA(n_components=10)

calcium_pca = pca_model.fit_transform(calcium_data)
# calcium_pca = pca_model.fit_transform(np.concatenate(normal_data))
color_list = ['red', 'green', 'blue', 'magenta']

# define the set of PCs to plot
pc = [0, 1, 2]

plot_list1 = []
plot_list2 = []
plot_list3 = []
for idx, el in enumerate(normal_data):

    if idx<8:
        c_idx = 0
    elif 8 < idx < 12:
        c_idx = 1
    elif 12 < idx < 20:
        c_idx = 2
    else:
        c_idx = 3
    
    plot_list1.append(hv.Curve((calcium_pca[trial_vector==idx, pc[0]], calcium_pca[trial_vector==idx, pc[1]])))
    plot_list2.append(hv.Curve((calcium_pca[trial_vector==idx, pc[0]], calcium_pca[trial_vector==idx, pc[2]])))
    plot_list3.append(hv.Curve((calcium_pca[trial_vector==idx, pc[1]], calcium_pca[trial_vector==idx, pc[2]])))
    
    plot_list1[idx].opts(width=600, height=600, color=hv.Cycle('Spectral'))
    plot_list2[idx].opts(width=600, height=600, color=hv.Cycle('Spectral'))
    plot_list3[idx].opts(width=600, height=600, color=hv.Cycle('Spectral'))
#     plot_list1[idx].opts(width=600, height=600, color=color_list[c_idx])
#     plot_list2[idx].opts(width=600, height=600, color=color_list[c_idx])
#     plot_list3[idx].opts(width=600, height=600, color=color_list[c_idx])
# proj2 = hv.Curve((calcium_pca[:, 0], calcium_pca[:, 2]))
# proj3 = hv.Curve((calcium_pca[:, 1], calcium_pca[:, 2]))

# proj1+proj2+proj3

ov1 = hv.Overlay(plot_list1)
ov2 = hv.Overlay(plot_list2)
ov3 = hv.Overlay(plot_list3)

(ov1+ov2+ov3).cols(1)
# plot the trajectory
# curve3d = hv.Path3D(calcium_pca)*hv.Scatter3D(np.reshape(calcium_pca[0,:],(1,-1)))
# curve3d

In [ ]:
hv.extension('bokeh')
# plot the PCA trajectories of the data on a per trial basis

# calcium_pca = pca_model.fit_transform(np.concatenate(normal_data))

plot_list1 = []
plot_list2 = []
plot_list3 = []
for idx, data in enumerate(normal_data):
    
    labels = list(data.columns)
    cells = [el for el in labels if 'cell' in el]
    # get the cell data
    calcium_data = np.array(data[cells].copy())
    # get rid of the super small values
    calcium_data[np.isnan(calcium_data)] = 0
    pca_model = decomp.PCA(n_components=3)

    calcium_pca = pca_model.fit_transform(calcium_data)

    plot_list1.append(hv.Curve((calcium_pca[:, 0], calcium_pca[:, 1])))
    plot_list2.append(hv.Curve((calcium_pca[:, 0], calcium_pca[:, 2])))
    plot_list3.append(hv.Curve((calcium_pca[:, 1], calcium_pca[:, 2])))
    
    plot_list1[idx].opts(width=600, height=600, color=hv.Cycle('Spectral'))
    plot_list2[idx].opts(width=600, height=600, color=hv.Cycle('Spectral'))
    plot_list3[idx].opts(width=600, height=600, color=hv.Cycle('Spectral'))
# proj2 = hv.Curve((calcium_pca[:, 0], calcium_pca[:, 2]))
# proj3 = hv.Curve((calcium_pca[:, 1], calcium_pca[:, 2]))

# proj1+proj2+proj3

ov1 = hv.Overlay(plot_list1)
ov2 = hv.Overlay(plot_list2)
ov3 = hv.Overlay(plot_list3)

(ov1+ov2+ov3).cols(1)

In [ ]:
# perform PSID

# define the target behavior(s)
# target_behavior = ['mouse_x', 'mouse_y', 'mouse_head_x', 'mouse_head_y', 
#                    'mouse_body2_x', 'mouse_body2_y', 'mouse_body3_x', 'mouse_body3_y', 
#                    'mouse_base_x', 'mouse_base_y', 'mouse_speed'
#                   , 'cricket_0_x', 'cricket_0_y','cricket_0_mouse_distance',
#                   'cricket_0_delta_heading', 'cricket_0_visual_angle']
# target_behavior = ['cricket_0_mouse_distance', 'cricket_0_x', 'cricket_0_y', 'cricket_0_visual_angle']

target_behavior = ['mouse_x', 'mouse_y', 'mouse_head_x', 'mouse_head_y', 
                   'mouse_body2_x', 'mouse_body2_y', 'mouse_body3_x', 'mouse_body3_y', 
                   'mouse_base_x', 'mouse_base_y', 'mouse_speed',
                   'cricket_0_mouse_distance', 'cricket_0_x', 'cricket_0_y', 
                   'cricket_0_delta_heading', 'cricket_0_speed']
# target_behavior = ['cricket_0_mouse_distance']
# target_behavior = ['mouse_x', 'mouse_y']
# 'cricket_0_x', 'cricket_0_y','cricket_0_mouse_distance'
# define the percentage of each trial to use as test
test_perc = 0.2

# allocate memory for the train and test ca and behavior data
beh_train = []
beh_test = []
ca_train = []
ca_test = []

# for all the trials
for idx, trial in enumerate(normal_data + mod_data):
#     # get the table
#     sub_data = pd.concat(normal_data)

    # get the available columns
    labels = list(trial.columns)
    cells = [el for el in labels if 'cell' in el]
    # get the cell data
    calcium_data = np.array(trial[cells].copy())
    # get rid of the super small values
    calcium_data[np.isnan(calcium_data)] = 0

    try:
        # get the parameter
        beh_data = trial[target_behavior].to_numpy()

        # smooth the parameter
        beh_data = ss.medfilt(beh_data, (21, 1))
    except KeyError:
        continue
    # filter the data
#     # remove the repeated parameter bins (due to redundancy)
#     keep_vector = np.pad(np.diff(beh_data[:, 0])!=0, (1, 0), mode='constant', constant_values=0)
    
#     # limit the parameter based on field of view
#     sec_parameter = trial.loc[:, 'cricket_0_delta_heading'].to_numpy()
#     parameter_range = (20 > sec_parameter)  #& (sec_parameter > -90)
#     keep_vector = (keep_vector) & (parameter_range)
    
    # limit the parameter based on distance to prey
#     distance_to_prey = trial.loc[:, 'cricket_0_mouse_distance'].to_numpy()
#     distance_vector = distance_to_prey > 3
#     keep_vector = (keep_vector) & (distance_vector)
    
#     # perform the removal
#     beh_data = beh_data[keep_vector, :]
#     calcium_data = calcium_data[keep_vector, :]
    # skip if empty
    if calcium_data.shape[0] == 0:
        continue
    
    downsamp = 1
    # bin the data
    if downsamp > 1:
        beh_data = ss.decimate(beh_data, downsamp, axis=0)
        calcium_data = ss.decimate(calcium_data, downsamp, axis=0)

    # get the threshold index
    threshold_idx = int(calcium_data.shape[0]*(test_perc))
    # split the data
    ca_trial_train = calcium_data[threshold_idx:, :] 
    ca_trial_test = calcium_data[:threshold_idx, :] 
    beh_trial_train = beh_data[threshold_idx:, :]
    beh_trial_test = beh_data[:threshold_idx, :] 
    
    # store the data
    ca_train.append(ca_trial_train)
    ca_test.append(ca_trial_test)
    beh_train.append(beh_trial_train)
    beh_test.append(beh_trial_test)

# scale the data
ca_scaler = preprocessing.StandardScaler().fit(np.concatenate(ca_train))
beh_scaler = preprocessing.StandardScaler().fit(np.concatenate(beh_train))

ca_train = [ca_scaler.transform(el) for el in ca_train]
ca_test = [ca_scaler.transform(el) for el in ca_test]
beh_train = [beh_scaler.transform(el) for el in beh_train]
beh_test = [beh_scaler.transform(el) for el in beh_test]

# train the PSID model
idSys = PSID.PSID(ca_train, beh_train, nx=8, n1=8, i=20)
# idSys = PSID.PSID(ca_train, beh_train, nx=1, n1=1, i=20) # for cricket distance
# idSys = PSID.PSID(ca_train, beh_train, nx=20, n1=10, i=35)

# allocate memory for the predictions
beh_pred = []
ca_pred = []
latent_pred = []
# predict each trial
for trial in ca_test:
    beh_p, ca_p, latent_p = idSys.predict(trial)
    beh_pred.append(beh_p)
    ca_pred.append(ca_p)
    latent_pred.append(latent_p)

combo_beh_test = np.vstack(beh_test)
combo_beh_pred = np.vstack(beh_pred)

combo_ca_test = np.vstack(ca_test)
combo_ca_pred = np.vstack(ca_pred)
    
R2TrialBased_beh = evalPrediction(combo_beh_test, combo_beh_pred, 'CC')
R2TrialBased_ca = evalPrediction(combo_ca_test, combo_ca_pred, 'CC')

print(np.sum(R2TrialBased_ca>0))
print(np.nanmean(R2TrialBased_ca))
print(R2TrialBased_beh)
# print(R2TrialBased_ca)




In [ ]:
# plot the PSID result compared to the variable of interest
hv.extension('bokeh')
combo_latent = np.vstack(latent_pred)

# get the x axis
x = np.array(np.arange(combo_beh_test.shape[0]))

c1 = hv.Curve((x, combo_beh_test[:, 0])).opts(width=800, title='Test vs predicted behavior')
c2 = hv.Curve((x, combo_beh_pred[:, 0]))

if combo_latent.shape[1] == 1:
    c3 = hv.Curve((x, combo_latent[:, 0])).opts(shared_axes=False, width=800, title='Latents')
else:
    c3 = hv.Image(combo_latent.T).opts(shared_axes=False, width=800, 
                                       cmap='viridis', tools=['hover'], title='Latents')
if combo_beh_test.shape[1] == 1:
    c4 = hv.Curve((x, combo_beh_test[:, 0])).opts(axiswise=True, 
                                                  width=800, tools=['hover'], title='Test behavior')
    c5 = hv.Curve((x, combo_beh_pred[:, 0])).opts(shared_axes=False, 
                                                  width=800, tools=['hover'], title='Predicted behavior')
else:
    c4 = hv.Image(combo_beh_test.T).opts(axiswise=True, 
                                         width=800, cmap='viridis', tools=['hover'], 
                                         title='Test behavior')
    c5 = hv.Image(combo_beh_pred.T).opts(shared_axes=False, 
                                         width=800, cmap='viridis', 
                                         tools=['hover'], title='Predicted behavior')
c6 = hv.Image(combo_ca_test.T).opts(shared_axes=False, 
                                    width=800, cmap='viridis', tools=['hover'], title='Test calcium')
c7 = hv.Image(combo_ca_pred.T).opts(shared_axes=False, 
                                    width=800, cmap='viridis', tools=['hover'], title='Predicted calcium')

(c1*c2+c3+c4+c5+c6+c7).cols(1)



In [ ]:
# run the model on all experiments

# allocate memory for the predictions
final_beh = []
final_ca = []
final_latent = []

# predict each trial
for trial in normal_data + mod_data:
    
    # get the available columns
    labels = list(trial.columns)
    cells = [el for el in labels if 'cell' in el]
    # get the cell data
    calcium_data = np.array(trial[cells].copy())
    # get rid of the super small values
    calcium_data[np.isnan(calcium_data)] = 0
    
    beh_p, ca_p, latent_p = idSys.predict(calcium_data)
    final_beh.append(beh_p)
    final_ca.append(ca_p)
    final_latent.append(latent_p)



In [ ]:
# plot the PSID dynamics reconstruction
hv.extension('plotly')
    
# opts.defaults(opts.Scatter3D(cmap='viridis'))
# define the target parameter
target_parameter = 'mouse_speed'
print(normal_data[0].columns[:40])

# define the target dimensions
tar_dim = np.array([0, 1, 2]) + 0

# allocate memory for the output list
plot_list = []

target_trial = [0, 8]
for idx, trial in enumerate(final_latent[target_trial[0]:target_trial[1]+1]):
# for idx, trial in enumerate(final_latent):
    
    # get a parameter
    try:
        parameter = normal_data[idx][target_parameter]
        parameter = (parameter-parameter.min())/(parameter.max()-parameter.min())
#         parameter = np.log(parameter)
    
        plot_list.append(hv.Scatter3D((trial[:, tar_dim[0]], trial[:, tar_dim[1]], trial[:, tar_dim[2]])))
        plot_list[idx].opts(size=2, height=800, width=800, color=parameter, colorbar=True)
#     plot_list[idx].opts(hv.opts.Scatter3D(cmap='viridis'))
    except KeyError:
#             parameter = np.zeros(trial.shape[0])
        continue

ov = hv.Overlay(plot_list)
ov
# hv.help(hv.Scatter3D)

In [ ]:
# plot individual latents


In [ ]:
#
# Embed with UMAP
transformed_data = np.concatenate(final_latent)
# embed the data via UMAP
reducer = umap.UMAP(min_dist=0.1, n_neighbors=30)
embedded_data = reducer.fit_transform(transformed_data)

In [ ]:
# Create the UMAP labels
# allocate memory for the distances
distance_list = []
target_key = 'cricket_0_mouse_distance'

# define the interval to take from the edges
# edges = [7, -14] # for bins = 14
# edges = [15, -15] # for bins = 30

# for all the files
for idx, files in enumerate(normal_data + mod_data):
#     temp_values = np.log(files.index[edges[0]:edges[1]])
#     temp_values = (files['time_vector'][edges[0]:edges[1]]/np.max(files['time_vector'][edges[0]:edges[1]]))
#     temp_values = files['mouse_speed'][edges[0]:edges[1]]
    if target_key in files.keys():
        temp_values = files[target_key]
#         temp_values = np.log(ss.medfilt(files[target_key], 41))
#         print(temp_values.shape)
#         temp_values = np.ones(files.shape[0])*idx
#         temp_values = np.log(files[target_key])
#         temp_values[temp_values>10] = 10

        temp_values[np.isinf(temp_values)] = 0
#         snout = files[['mouse_snout_x', 'mouse_snout_y']].to_numpy()[edges[0]:edges[1], :]
#         tail = files[['mouse_base_x', 'mouse_base_y']].to_numpy()[edges[0]:edges[1], :]
#         temp_values = fk.distance_calculation(snout, tail)
#         temp_values[temp_values>10] = 10

    else:
#         temp_values = np.zeros_like(files['mouse_x'][edges[0]:edges[1]])
        temp_values = np.zeros_like(files['mouse_x'])
#     temp_values[temp_values==0] = np.nan
    distance_list.append(temp_values)

In [ ]:
# plot the UMAP results
hv.extension('bokeh')

# define the interval between points
interv = 2

compiled_labels = np.expand_dims(np.hstack(distance_list), axis=1)

umap_data = np.concatenate((embedded_data,compiled_labels),axis=1)

compiled_labels = compiled_labels[::interv]
umap_data = umap_data[::interv, :]


umap_plot = hv.Scatter(umap_data, vdims=['Dim 2',target_key], kdims=['Dim 1'])
umap_plot.opts(color=target_key, colorbar=True, cmap='Spectral', size=3)

umap_plot.opts(height=600, width=800)

umap_plot

# umap_plot.opts(opts.Scatter(width=fp.pix(5.7), height=fp.pix(7.8), toolbar=None, 
#                         hooks=[fp.margin], fontsize=fp.font_sizes['small'], xticks=3, yticks=3))

In [ ]:
# Calculate pairwise distances

# define the behavioral variables
# target_behavior = ['cricket_0_mouse_distance', 'cricket_0_speed', 'cricket_0_visual_angle',
#                   'cricket_0_delta_heading']

target_behavior = ['mouse_x', 'mouse_y', 'mouse_head_x', 'mouse_head_y', 
                   'mouse_body2_x', 'mouse_body2_y', 'mouse_body3_x', 'mouse_body3_y', 
                   'mouse_base_x', 'mouse_base_y', 'mouse_speed']
# define the distance metric
distance_metric = 'manhattan'

# allocate memory for the distances
distance_list = []
# for all the trials
for trial in normal_data:
    # get the behavioral variables
    behavior = trial[target_behavior].to_numpy()
    # get the calcium data
    labels = list(sub_data.columns)
    cells = [el for el in labels if 'cell' in el]
    calcium = trial[cells].to_numpy()
    
    # downsample
    downsamp = 2
    # bin the data
    if downsamp > 1:
        behavior = ss.decimate(behavior, downsamp, axis=0)
        calcium = ss.decimate(calcium, downsamp, axis=0)
    
    # calculate the respective distances
    behavior_distance = smet.pairwise_distances(behavior, metric=distance_metric)
    behavior_distance = behavior_distance[np.triu_indices(behavior_distance.shape[0], 1)]
    calcium_distance = smet.pairwise_distances(calcium, metric=distance_metric)
    calcium_distance = calcium_distance[np.triu_indices(calcium_distance.shape[0], 1)]
    # store
    distance_list.append((behavior_distance, calcium_distance))

In [ ]:
# plot the distances

# allocate the plot list
plot_list = []
# define the density of sampling
sampling = 10
# for every trial
for trial in distance_list:
    
#     curr_plot = hv.Scatter((trial[0][::sampling], trial[1][::sampling]))
#     curr_plot.opts(width=800, height=800, logx=True)
#     x = trial[0][::sampling]
#     y = trial[1][::sampling]
    
    x = np.log(trial[0][::sampling])
    y = np.log(trial[1][::sampling])
    x[np.isinf(x)] = 0
    y[np.isinf(y)] = 0
    
#     np.random.shuffle(y)

    H, xedges, yedges = np.histogram2d(x, y, 100)
    curr_plot = hv.Image(np.flipud(H))
    curr_plot.opts(height=400, width=400, cmap='Viridis')
    plot_list.append(curr_plot)

# plot
hv.Layout(plot_list).cols(2)

In [ ]:
hv.extension('bokeh')
# im_real = hv.Image(calcium_data.T).opts(width=800, cmap='viridis', tools=['hover'])
# im_real
target_behavior = ['mouse_x', 'mouse_y', 'mouse_head_x', 'mouse_head_y', 
                   'mouse_body2_x', 'mouse_body2_y', 'mouse_body3_x', 'mouse_body3_y', 
                   'mouse_base_x', 'mouse_base_y', 'mouse_speed']
# hv.Curve(calcium_data[:, 0])
all_behavior = np.concatenate([el[target_behavior].to_numpy() for el in normal_data], axis=0)
print(all_behavior.shape)
all_behavior = np.array([(el-np.nanmin(el))/(np.nanmax(el)-np.nanmin(el)) for el in all_behavior]).T

hv.Image(all_behavior).opts(width=800, cmap='viridis').redim.range(z=(0, 0.9))

# print(np.sum(np.isnan(calcium_data)))
# im_last = hv.Image(calcium_last.T).opts(width=800, cmap='viridis')

# im_combo = (im_real + im_last).cols(1)
# im_combo

In [ ]:
# Plot single cell 

all_calcium

hv.Image(all_behavior).opts(width=800, cmap='viridis').redim.range(z=(0, 0.9))


In [ ]:
# Calculate the per cell rev correlation

# define the interval for calculation in frames
target_interval = 10
# define the target behavioral variable
# target_behavior = 'cricket_0_mouse_distance'
target_behavior = ['mouse_x', 'mouse_y', 'mouse_head_x', 'mouse_head_y', 
                   'mouse_body2_x', 'mouse_body2_y', 'mouse_body3_x', 'mouse_body3_y', 
                   'mouse_base_x', 'mouse_base_y', 'mouse_speed',
                   'cricket_0_x', 'cricket_0_y','cricket_0_mouse_distance',
                   'cricket_0_delta_heading', 'cricket_0_visual_angle']

# get the behavioral data
# current_behavior = np.expand_dims(data.loc[:, target_behavior].to_numpy(), axis=1)
current_behavior = data.loc[:, target_behavior].to_numpy()
beh_columns = len(target_behavior)

# concatenate all the data
data = pd.concat(normal_data, axis=0)

# get the available columns
labels = list(data.columns)
cells = [el for el in labels if 'cell' in el]
# get the cell data
calcium_data = np.array(data[cells].copy()).T
# scale for min and max
calcium_data = (calcium_data - np.nanmin(calcium_data))/\
               (np.nanmax(calcium_data) - np.nanmin(calcium_data))
# get rid of the nan values
calcium_data[np.isnan(calcium_data)] = 0
# get the number of time points
time_number = calcium_data.shape[1]
# get the number of cells
roi_number = calcium_data.shape[0]

# pad the behavior data
# padded_behavior = np.vstack((np.zeros((target_interval, 1)), current_behavior)).T

padded_behavior = np.vstack((np.zeros((target_interval, beh_columns)), current_behavior, 
                             np.zeros((target_interval, beh_columns)))).T
# remove nans (heads up, there are a lot)
padded_behavior[np.isnan(padded_behavior)] = 0

# allocate memory to save the ols
ols_list = []
prediction_list = []
r2_list = []
# for all the cells
for cells in np.arange(roi_number-36):
    print(f'current cell: {cells}')
    # allocate memory for the padded matrix
#     time_matrix = np.zeros((time_number, target_interval))
    time_matrix = np.zeros((time_number, 2*target_interval*beh_columns))
    # get the calcium data
    current_calcium = calcium_data[cells:cells+1, :].T
        
    # fill the padded matrix
    for frame, el in enumerate(current_calcium):
#         time_matrix[frame, :] = padded_behavior[:, frame:frame+target_interval]
        time_matrix[frame, :] = padded_behavior[:, frame:frame+2*target_interval].flatten()
#     # calculate the covariance matrix
#     cov_matrix = time_matrix.T@time_matrix
#     # calculate the STA
#     sta = time_matrix.T@current_calcium
#     try:
#         # calculate the ols estimate
#         ols = np.linalg.inv(cov_matrix) @ sta   
#     except np.linalg.LinAlgError:
#         ols = []
        
# ols_list.append(ols)
#     ols_estimator = lin.PoissonRegressor(alpha=10, max_iter=5000, fit_intercept=False)
    ols_estimator = lin.TweedieRegressor(alpha=0, max_iter=10000, fit_intercept=True, power=1)
    ols_estimator.fit(time_matrix, current_calcium.ravel())
    # save the ols
    ols_list.append(ols_estimator.coef_)
    current_prediction = ols_estimator.predict(time_matrix)
    print(current_prediction)
    prediction_list.append(current_prediction)
    r2_list.append(smet.r2_score(current_calcium, current_prediction))
    

In [ ]:
# plot the input data
print(current_behavior.shape)
hv.extension('bokeh')
im = hv.Image(current_behavior[:10000, :].T).opts(width=1000, height=400)


im

In [ ]:
# plot the fit quality

freq, edges = np.histogram(r2_list)
hv.Bars((edges, freq)).opts(width=800, tools=['hover'])

In [ ]:
# Plot the OLS

# allocate memory for the cells
cell_dict = {}
prediction_dict = {}

# generate the x axis vector
# x = range(target_interval)
x_pred = range(current_behavior.shape[0])
x = range(2*target_interval)
# for all the cells
for cells in np.arange(roi_number-36):
    
    # generate the plot and save
#     cell_plot = hv.Curve((x, ols_list[cells].flatten()), kdims=['Time'], vdims=target_behavior)
#     cell_plot.opts(shared_axes=False, width=600)
    
    current_calcium = calcium_data[cells:cells+1, :].ravel()

    prediction_plot = hv.Curve((x_pred, current_calcium)) * \
        hv.Curve((x_pred, prediction_list[cells]))
    prediction_plot.opts(shared_axes=False, width=600)
    # save in a dict
#     cell_dict[cells] = cell_plot
    prediction_dict[cells] = prediction_plot

cell_holomap = hv.HoloMap(prediction_dict)
cell_holomap

In [ ]:
# train on the mod data

# define the target variable
# target_behavior = ['cricket_0_x', 'cricket_0_y']
# target_behavior = ['mouse_x', 'mouse_y']
target_behavior = ['cricket_0_mouse_distance']
direct_label = 0
# define the number of bins
bin_number = 5
# print(str(mouse)+str(day))
# get the table
# sub_data = data[mouse][day]
sub_data = mod_data
# get the available columns
labels = list(sub_data.columns)
cells = [el for el in labels if 'cell' in el]
not_cells = [el for el in labels if 'cell' not in el]
# get the cell data
calcium_data_mod = np.array(sub_data[cells].copy())
# get rid of the super small values
calcium_data_mod[np.isnan(calcium_data_mod)] = 0

# if calcium_data.shape[0] == 0:
#     continue
# scale (convert to float to avoid warning, potentially from using too small a dtype)
calcium_data_mod = preprocessing.StandardScaler().fit_transform(calcium_data_mod)

# get the distance to cricket
# distance = ss.medfilt(sub_data.loc[:, target_behavior].to_numpy(), 21)
parameter = sub_data.loc[:, target_behavior].to_numpy()
keep_vector = np.pad(np.diff(parameter[:, 0])!=0, (1, 0), mode='constant', constant_values=0)

# distance_to_prey = sub_data.loc[:, 'cricket_0_mouse_distance'].to_numpy()
# distance_vector = distance_to_prey>3
# keep_vector = (keep_vector) & (distance_vector)

parameter = parameter[keep_vector, :]
calcium_data_mod = calcium_data_mod[keep_vector, :]

# random.shuffle(parameter)


# # shufle the label vector
# random.shuffle(label_vector)
# random.shuffle(distance)

# train the classifier

calcium_train, calcium_test, parameter_train, parameter_test = \
    train_test_split(calcium_data_mod, parameter, test_size=0.2)

linear_mod = lin.MultiTaskElasticNetCV(max_iter=5000, l1_ratio=[.1, .5, .7, .9, .95, .99, 1], n_jobs=7)
linear_mod.fit(calcium_train, parameter_train)
linear_pred_mod = linear_mod.predict(calcium_test)


print(calcium_data_mod.shape)


print('Exp variance:'+str(smet.r2_score(parameter_test, linear_pred_mod)))
print(linear_mod.alpha_)
print(linear_mod.l1_ratio_)

In [ ]:
# predict dark position with light decoder

linear_pred_mod = linear_mod.predict(calcium_data_mod)
linear_pred_combo = linear.predict(calcium_data_mod)

In [ ]:
# Plot the prediction over time

# get the number of time points
time_number = linear_pred.shape[0]
# define the x axis
x_axis = np.array(range(time_number))
# print(time_number)

# normalized_param = parameter_test
parameter = mod_data.loc[:, target_behavior].to_numpy()

# normalized_param = normalized_param/np.max(normalized_param)

# normalized_prediction = ss.medfilt(bins[linear_pred], 21)
# normalized_prediction = ss.medfilt(bins[linear_pred], 21)
# normalized_prediction = ss.medfilt(linear_pred, 21)
# normalized_prediction = linear_pred.astype(float)
normalized_prediction = ss.medfilt(linear_pred.astype(float), (1, 1))

# normalized_prediction = normalized_prediction/np.max(normalized_prediction)
# normalized_prediction = ss.medfilt(linear_pred, 21)

# normalized_labels = ss.medfilt(bins[label_vector], 21)

# quadrant_info = sub_data.loc[keep_vector, 'cricket_0_quadrant']
quadrant_info = sub_data.loc[:, 'cricket_0_quadrant'].to_numpy()
quadrant_info = quadrant_info[keep_vector]
# quadrant_info = ss.medfilt(quadrant_info/np.max(quadrant_info),21)

distance_info = distance_to_prey[keep_vector]
# # filter the prediction by the quadrant
# normalized_vis = normalized_prediction.copy()
# normalized_vis[quadrant_info>0] = np.nan
# normalized_nonvis = normalized_prediction.copy()
# normalized_nonvis[quadrant_info<1] = np.nan

# angle_info = sub_data.loc[keep_vector, 'cricket_0_visual_angle']
# angle_info = ss.medfilt(angle_info/np.max(angle_info),21)

# speed_info = sub_data.loc[keep_vector, 'mouse_speed']
# speed_info = ss.medfilt(speed_info/np.max(speed_info),21)

heading_info = sub_data.loc[:, 'cricket_0_delta_heading'].to_numpy()
heading_info = heading_info[keep_vector]

# prediction = hv.Curve((range(time_number), normalized_prediction)).opts(height=600, width=800)
# prediction_vis = hv.Curve((range(time_number), normalized_vis)).opts(height=600, width=800)
# prediction_nonvis = hv.Curve((range(time_number), normalized_nonvis)).opts(height=600, width=800)
parameter1 = hv.Curve((x_axis, normalized_param[:, 0])).opts(height=300, width=800)
parameter2 = hv.Curve((x_axis, normalized_param[:, 1])).opts(height=300, width=800)

prediction1 = hv.Curve((x_axis, normalized_prediction[:, 0])).opts(height=300, width=800)
prediction2 = hv.Curve((x_axis, normalized_prediction[:, 1])).opts(height=300, width=800)
# labels = hv.Curve((range(time_number), normalized_labels)).opts(height=600, width=800)
mse_info = np.sqrt((normalized_param - normalized_prediction)**2)
mse1 = hv.Curve((x_axis, mse_info[:, 0])).opts(height=300, width=800)
# quadrant = hv.Curve((range(time_number), quadrant_info)).opts(height=600, width=800)
# quadrant = hv.Spikes(x_axis[quadrant_info==0]).opts(alpha=.3, spike_length=40)
# distance = hv.Curve((x_axis, distance_info))
# heading = hv.Curve((x_axis, heading_info))
# angle = hv.Curve((range(time_number), angle_info)).opts(height=600, width=800, tools=['hover'])
# speed = hv.Curve((range(time_number), speed_info)).opts(height=600, width=800)

# prediction+parameter
# compiled = parameter*prediction#*angle*speed
# compiled = parameter*prediction_vis*prediction_nonvis
# compiled = mse1*quadrant*distance*heading + parameter2*prediction2*quadrant
# compiled = parameter1*prediction1 + parameter2*prediction2
scatter_x = hv.Scatter((normalized_param[:, 0], normalized_prediction[:, 0]), kdims=['Real', 'Predicted'])
scatter_y = hv.Scatter((normalized_param[:, 1], normalized_prediction[:, 1]), kdims=['Real', 'Predicted'])
compiled = parameter1*prediction1 + parameter2*prediction2 + scatter_x + scatter_y
compiled.cols(1)
compiled